In [ ]:
from torch import cuda
from accelerate import infer_auto_device_map, init_empty_weights
import transformers
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM

# model_id = 'meta-llama/Llama-2-7b-chat-hf'
# model_id = 'openlm-research/open_llama_3b_v2'
# model_id = 'princeton-nlp/Sheared-LLaMA-2.7B'
# model_id = 'gpt2'

# device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
# device

In [ ]:
# begin initializing HF items, need auth token for these
model_config = AutoConfig.from_pretrained(
    model_id,
    # use_auth_token=hf_auth
)

with init_empty_weights():
    model = AutoModelForCausalLM.from_config(model_config)
model

In [ ]:
device_map = infer_auto_device_map(model, no_split_module_classes=["LlamaDecoderLayer"])
device_map

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    # quantization_config=bnb_config,
    device_map='auto',
    # use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    # use_auth_token=hf_auth
)

In [ ]:
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    temperature=0.0,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # mex number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

In [ ]:
res = generate_text("Explain to me the difference between nuclear fission and fusion.")
print(res[0]["generated_text"])